#### TODO

1. Refine the positions and the radius of the spheres
2. Try IK for Talos?

# Ik of Panda robot with SDF cost function

In [2]:
import sys
sys.path.append('../lib')

from pb_utils import set_q, get_joint_limits
from ocp_utils import init_pybullet, create_primitives

import pybullet as p
import pybullet_data

import time
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import clear_output
from scipy.optimize import minimize
from functools import partial

%load_ext autoreload
%autoreload 2
np.set_printoptions(precision=4, suppress=True)

## Start pybullet

In [3]:
p.connect(p.GUI)

0

#### Load the SDF

In [4]:
data = np.load('data/sdf.npy', allow_pickle=True)[()]
sdf_matr = data['sdf_matr']  #SDF tensor
obstacles = data['obstacles'] #obstacles parameters
dL = data['dL'] # Env discretization
L = data['L']   # Env length
N = data['N']   # Number of discretization for each axis
V = data['V']   # Voxel coordinates
V_flat = data['V_flat']  # Flattened vector of V
V_vec = V[0,0,:,2] #only valid for cube environment. Othervise, calculate vec for each of the axis
bounds = data['bounds'] # Bound of the environment

#### Visualize obstacles in pybullet

In [5]:
colors = [[0.8, 0.5, 0.5, 1]]*len(obstacles)
obj_id, init_id, target_id, border_id, obstacle_ids = init_pybullet (np.zeros(3), np.zeros(3), obstacles, colors=colors)

#### Visualize the SDF as mesh

#### Functions to access the sdf values

In [6]:
def get_indices(x):
    '''
    Given a coordinate x, find the set of indices 
    in the sdf tensor that corresponds to x
    '''
    dist = (x[:,None] - V_vec)**2
    indices = np.argmin(dist, axis=1)
    return indices[0], indices[1], indices[2]

def remove_border(idx, N=100):
    '''
    At the border, change the index to the one
    before the border, so we can still get the numdiff gradient
    Set N to be the same as the number of discretization in the voxel
    '''
    if idx == 0:
        return 1
    if idx == N-1:
        return idx-1
    return idx 

def compute_sdf(x):
    '''
    Calculate the sdf value at coordinate x
    '''
    x = np.array(x)
    idx1, idx2, idx3 = get_indices(x)
    sdf_val = sdf_matr[idx1][idx2][idx3]
    sdf_val_grad = compute_grad_sdf(x)

    #interpolate sdf val as f(x*) + f'(x*)(x-x*)
    #where x* is the closest in V to x
    dx = x - V[idx1][idx2][idx3]
    sdf_val += sdf_val_grad.dot(dx)
    return sdf_val

def compute_grad_sdf(x, inc = 1):
    '''
    Calculate the gradient of the sdf at x with numerical differentiation
    inc is 1 to get the neighbour voxel
    '''
    
    idx1, idx2, idx3 = get_indices(x)
    
    #modify at the border ***to be changed***
    idx1 = remove_border(idx1)
    idx2 = remove_border(idx2)
    idx3 = remove_border(idx3)
    
    #computing df_x
    sdf_val_p = sdf_matr[idx1][idx2][idx3+inc]
    sdf_val_m = sdf_matr[idx1][idx2][idx3-inc]
    dfz = (sdf_val_p-sdf_val_m)/(2*inc*dL)

    #computing df_y
    sdf_val_p = sdf_matr[idx1][idx2+inc][idx3]
    sdf_val_m = sdf_matr[idx1][idx2-inc][idx3]
    dfy = (sdf_val_p-sdf_val_m)/(2*inc*dL)
    
    #computing df_z
    sdf_val_p = sdf_matr[idx1+inc][idx2][idx3]
    sdf_val_m = sdf_matr[idx1-inc][idx2][idx3]
    dfx = (sdf_val_p-sdf_val_m)/(2*inc*dL)

    return np.array([dfx, dfy, dfz])

def barrier_obs(d, margin = 0.1):
    '''
    Set the value to be 0 if d > margin, and |d - margin| otherwise
    '''
    if d-margin < 0:
        return np.abs(d-margin)
    else:
        return 0

#### Load the robot

In [7]:
# p.resetSimulation()
DATA_PATH = '../data'
robot_urdf = DATA_PATH + '/urdf/frankaemika_new/panda_arm.urdf'
robot_id = p.loadURDF(fileName=robot_urdf)
joint_limits = get_joint_limits(robot_id, np.arange(7))

In [8]:
set_q_std = partial(set_q, robot_id, np.arange(7))

#### Base placement

In [9]:
p0 = np.array([0,-0.5,-0.15])
# p0 = np.zeros(3)
p.resetBasePositionAndOrientation(robot_id, p0, (0,0,0,1))

#### Check FK

In [10]:
import numpy as jnp
def TransformationToPose(T):
    R = T[:3,:3]
    x=T[:3,-1]
    sy = jnp.sqrt(R[0,0] * R[0,0] +  R[1,0] * R[1,0])
    singular = sy < 1e-6
    t1 = jnp.array([jnp.arctan2(R[2,1] , R[2,2]),jnp.arctan2(-R[2,0], sy),jnp.arctan2(R[1,0], R[0,0])]) 
    t2 = jnp.array([jnp.arctan2(-R[1,2], R[1,1]),jnp.arctan2(-R[2,0], sy),0])
    ts = jnp.where(singular, t2, t1)
    return jnp.concatenate((x,ts))

    
def compute_transformation(a=jnp.array([0,0,0,0.0825,-0.0825,0,0.088,0]),
                           d=jnp.array([0.333,0,0.316,0,0.384,0,0,0.107]),
                           alpha=jnp.array([0,-1,1,1,-1,1,1,0])*(jnp.pi/2)):
    def get_Ti(a_i,d_i,alpha_i):
        ca= jnp.cos(alpha_i)
        sa= jnp.sin(alpha_i)
        Talpha = jnp.array([[1,0,0,0],[0,ca,-sa,0],[0,sa,ca,0],[0,0,0,1]])
        Ta = jnp.array([[1,0,0,a_i],[0,1,0,0],[0,0,1,0],[0,0,0,1]])
        Td = jnp.array([[1,0,0,0],[0,1,0,0],[0,0,1,d_i],[0,0,0,1]])
        def T_i(theta_i):
            ct = jnp.cos(theta_i)
            st = jnp.sin(theta_i)
            Tz = jnp.array([[ct,-st,0,0],[st,ct,0,0],[0,0,1,0],[0,0,0,1]])
            return Talpha@Ta@Td@Tz
        
        return T_i

    T_all = []
    for i in range(len(a)):
        T_all.append(get_Ti(a[i],d[i],alpha[i]))
    def compute_T(theta):
        T = jnp.eye(4)
        Ts = []
        theta = jnp.concatenate((theta,jnp.array([0])))
        for i in range(len(theta)):
            T = T@T_all[i](theta[i])
            Ts.append(T)
        return T,Ts
    return compute_T

In [11]:
compute_T = compute_transformation()

### Create collision spheres for the body

In [12]:
class BodySphere():
    def __init__(self, pb_id, radius, joint_id, pos, abs_pos = None):
        self.radius = radius #radius of the sphere
        self.joint_id = joint_id #the id of the parent joint
        self.pos = pos #relative position w.r.t. the parent joint
        self.pb_id = pb_id #pybullet index of the corresponding object
        self.abs_pos = abs_pos #absolute position, w.r.t the base coordinate
        
    def set_abs_pos(self, abs_pos):
        '''
        Update the absolute position
        '''
        self.abs_pos = abs_pos 
        
def update_pos_spheres(spheres, Ts, offset=np.zeros(3)):
    '''
    Given the transformation matrices Ts, 
    update the absolute position of each sphere
    '''
    spheres_pos = []
    for i, sphere in enumerate(spheres):
        R, p0 = Ts[sphere.joint_id][:3,:3], Ts[sphere.joint_id][:3,3] 
        pos = p0 + R.dot(sphere.pos) 
        sphere.set_abs_pos(pos+offset)
    return

def visualize_pos_spheres(spheres):
    '''
    Update the sphere position in pybullet
    '''
    for i, sphere in enumerate(spheres):
        p.resetBasePositionAndOrientation(sphere.pb_id, sphere.abs_pos, (0,0,0,1))
    return

In [13]:
#define the radius of each sphere
body_radius = np.array([0.08]*12)

#create the object in pybullet
ball_ids = []
for i in range(len(body_radius)):
    _,_,ball_id = create_primitives(radius=body_radius[i], rgbaColor=[1,1,0,0.7])
    ball_ids.append(ball_id)

In [14]:
#create the collision sphere objects
sphere1 = BodySphere(pb_id = ball_ids[0], radius = body_radius[0], joint_id = 1, pos = np.array([0,0,0.05]))
sphere2 = BodySphere(pb_id = ball_ids[1], radius = body_radius[0], joint_id = 1, pos = np.array([-0.0,-0.1,0.01]))
sphere3 = BodySphere(pb_id = ball_ids[2], radius = body_radius[0], joint_id = 2, pos = np.array([0,-0.,-0.1]))
sphere4 = BodySphere(pb_id = ball_ids[3], radius = body_radius[0], joint_id = 3, pos = np.array([0,-0.,-0.]))
sphere5 = BodySphere(pb_id = ball_ids[4], radius = body_radius[0], joint_id = 3, pos = np.array([-0.1,0.1,-0.]))
sphere6 = BodySphere(pb_id = ball_ids[5], radius = body_radius[0], joint_id = 4, pos = np.array([0,0,0.0]))
sphere7 = BodySphere(pb_id = ball_ids[6], radius = body_radius[0], joint_id = 4, pos = np.array([0,0.1,-0.]))
sphere8 = BodySphere(pb_id = ball_ids[7], radius = body_radius[0], joint_id = 4, pos = np.array([0,0.1,-0.1]))
sphere9 = BodySphere(pb_id = ball_ids[8], radius = body_radius[0], joint_id = 4, pos = np.array([0,0.,-0.2]))
sphere10 = BodySphere(pb_id = ball_ids[9], radius = body_radius[0], joint_id = 5, pos = np.array([0.1,-0.,-0.]))
sphere11 = BodySphere(pb_id = ball_ids[10], radius = body_radius[0], joint_id = 6, pos = np.array([0.,-0.,-0.]))
sphere12 = BodySphere(pb_id = ball_ids[11], radius = body_radius[0], joint_id = 7, pos = np.array([0.,-0.,-0.]))
body_spheres = [sphere1, sphere2, sphere3, sphere4, sphere5, sphere6, sphere7, sphere8, sphere9, 
               sphere10, sphere11, sphere12]

#### Test the FK function

In [15]:
q = joint_limits[0] + np.random.rand(7)*(joint_limits[1]-joint_limits[0])
T, Ts = compute_T(q)
set_q_std(q)

In [16]:
update_pos_spheres(body_spheres, Ts, offset = p0 + np.array([0,0,-0.05]))
visualize_pos_spheres(body_spheres)

#### IK as optimization

#### Note for Tobias:
- shift the FK result by the robot base
- 0.05 shift in pybullet (OK)
- global variables for the cost (pos_target, sdf_matr, body_spheres)

In [18]:
def calc_cost(q, w_obs = 10):
    T, Ts = compute_T(q)
    pos = T[:3,3] + p0
    err = np.linalg.norm(pos-pos_target)
    update_pos_spheres(body_spheres, Ts, offset = +p0 + np.array([0,0,-0.05]))
    for sphere in body_spheres:
        cost = w_obs*barrier_obs(compute_sdf(sphere.abs_pos), margin=sphere.radius)
        err += cost
    return err

### Solve using scipy

In [19]:
pos_target = np.array([0.7,-0.5,0.3])
p.resetBasePositionAndOrientation(target_id, pos_target, (0,0,0,1))

#### Initialize using N random samples and pick the best one

In [25]:
qs = joint_limits[0] + np.random.rand(4000,7)*(joint_limits[1]-joint_limits[0])

costs = []
for i,q in enumerate(qs):
    costs.append(calc_cost(q))
idx = np.argmin(costs)
q0 = qs[idx]
print(costs[idx])

0.0937122503855981


#### Initialize using random sample

In [24]:
q0 = joint_limits[0] + np.random.rand(7)*(joint_limits[1]-joint_limits[0])
set_q_std(q0)

#### Solve

In [26]:
def callback(xk):
    print(calc_cost(xk.flatten()))
    return

res = minimize(calc_cost, q0, 
               method='BFGS', options={'disp':True, 'maxiter':1000}, callback=callback)

0.014605454038706528
0.010618394046948625
0.003051254123573261
0.0013490459644987611
0.0007027601191039007
0.0004361512085080782
0.00031968438876656824
0.0001648102673402822
9.103637774869425e-05
4.644126133048379e-05
3.0338375961750664e-05
2.6156171127868723e-05
1.857039125343312e-05
6.041505166607212e-06
3.6230700756483928e-06
2.760326071040622e-06
1.8548707312242082e-06
1.0631118087038472e-06
1.0036776449904113e-06
         Current function value: 0.000001
         Iterations: 19
         Function evaluations: 644
         Gradient evaluations: 79


#### Visualize the result

In [27]:
q = res['x']
set_q_std(q)

T, Ts = compute_T(q)
update_pos_spheres(body_spheres, Ts, offset=p0 + np.array([0,0,-0.05]))
visualize_pos_spheres(body_spheres)